place holder

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import os, sys
sys.path.append(os.path.abspath("../src"))

from data_pipeline.builders import (
    build_market_daily, build_market_plus_hvol_fwd,
    build_options_snapshot, build_vol_surface_long
)

mkt   = build_market_daily()
mktx  = build_market_plus_hvol_fwd()
opts  = build_options_snapshot()
vsurf = build_vol_surface_long()

mkt.tail(), mktx.tail(), opts.head(), vsurf.head()


ModuleNotFoundError: No module named 'data_pipeline'